In [1]:
# Setup
# `jupyter nbextension enable --py --sys-prefix ipyvuetify`#
# start: `voila .\smart_grids.ipynb`
# icons here https://vuetifyjs.com/en/features/icon-fonts/#material-design-icons


In [2]:
import ipywidgets
from matplotlib import pyplot as plt
import numpy as np
import ipyvuetify as v
from util.get_photovoltaic_data import get_photovoltaic_data
from loguru import logger


In [3]:
def update_stepper(widget, *args):
    stepper.v_model += widget.direction
    try:
        widget.step.complete = True
    except:
        pass


In [4]:
class BackBtn(v.Btn):
    def __init__(self):
        self.direction = -1
        super().__init__(children="Back")
        self.on_event("click", update_stepper)


In [5]:
class NextBtn(v.Btn):
    def __init__(self, container, step, disabled=True):
        self.step = step
        self.direction = 1
        self.container = container
        container.next_btn = self
        super().__init__(children="Next")
        self.on_event("click", update_stepper)
        self.disabled = disabled


In [6]:
import address_widget

address_container = address_widget.Address()
# address_container
address_step_number = 1
address_step = v.StepperStep(
    step=address_step_number, children=["Address"], complete=False
)
address_step_content = v.StepperContent(
    step=address_step_number,
    children=[address_container, NextBtn(address_container, address_step)],
)


In [7]:
import battery_widget

battery_container = battery_widget.Battery()
# battery_container
battery_step_number = 2
battery_step = v.StepperStep(
    step=battery_step_number, children=["Battery"], complete=False
)
battery_step_content = v.StepperContent(
    step=battery_step_number,
    children=[battery_container, BackBtn(), NextBtn(battery_container, battery_step)],
)


In [33]:
import pandas as pd
import numpy as np


def convert_pv_data_to_dataframe(data):
    HOURS_IN_DAY = 24
    df = {"day": []}  # "date": [],
    for i in range(0, 24):
        df.setdefault(i, [])

    for day in range(0, len(data), HOURS_IN_DAY):
        hours = data[day : day + HOURS_IN_DAY]
        # df["date"].append(hours[0]["time"].split(":")[0])
        df["day"].append(day / 24 + 1)
        for i, d in enumerate(hours):
            df[i].append(d["PV system power"])

    return pd.DataFrame(df).set_index("day").rename_axis("hour",axis="columns")


In [36]:
def get_pv_data():
    response = get_photovoltaic_data(
        latitude=address_container.latitude.v_model,
        longitude=address_container.longitude.v_model,
        pv_tech=pv_container.pv_technology.v_model,
        pv_peak_power=pv_container.pv_peak_power.v_model,
        pv_loss=pv_container.pv_loss.v_model,
        pv_angle=pv_container.pv_slope.v_model,
        pv_azimuth=pv_container.pv_azimuth.v_model,
        start_year=2015,
        end_year=2015,
    )
    pv_data = [
        {"time": data["time"], "PV system power": data["P"]}
        for data in response["outputs"]["hourly"]
    ]

    return convert_pv_data_to_dataframe(pv_data)


In [37]:
# get_pv_data()

2022-07-10 20:44:31.326 | INFO     | util.get_photovoltaic_data:get_photovoltaic_data:33 - query: https://re.jrc.ec.europa.eu/api/seriescalc?lat=50.11207962036133&lon=8.683409690856934&horirrad=1&outputformat=json&pvcalculation=1&startyear=2015&endyear=2015&peakpower=4&pvtechchoice=CIS&angle=45&aspect=180&loss=0


hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.20,246.68,...,72.04,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.28,51.36,...,97.48,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,70.52,126.64,...,55.76,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,74.52,217.88,...,38.84,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,71.28,144.00,...,123.88,23.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63.20,211.64,...,119.76,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
362.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.16,224.96,...,155.24,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
363.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,77.48,148.72,...,147.08,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
import pv_widget

pv_container = pv_widget.Photovoltaic()
pv_step_number = 3
pv_step = v.StepperStep(step=pv_step_number, children=["Photvoltaic"], complete=False)
pv_calc_btn = v.Btn(children=["calculate"],disabled=True)
pv_container.calc_btn=pv_calc_btn
pv_next_button=NextBtn(pv_container, pv_step)

def calculate_pv_power(widget, *args):
    pv_container.pv_card.loading = True
    pv_container.pv_power_data = get_pv_data()
    pv_step.complete = True
    pv_container.children=[pv_container.pv_card]+[pv_container.get_fig()]
    pv_next_button.disabled=False

    pv_container.pv_card.loading = False

pv_calc_btn.on_event("click", calculate_pv_power)
pv_step_content = v.StepperContent(
    step=pv_step_number, children=[pv_container, BackBtn(),pv_calc_btn, pv_next_button]
)


In [11]:
# pv_power_container = v.Container(children=[get_data_fig()])
# pv_power_step_number = 4
# pv_power_step = v.StepperStep(
#     step=pv_power_step_number, children=["PV Power"], complete=False
# )
# pv_power_step_content = v.StepperContent(
#     step=pv_power_step_number,
#     children=[
#         pv_power_container,
#         BackBtn(),
#         NextBtn(pv_power_container, pv_power_step, disabled=False),
#     ],
# )


In [12]:
# import consumers_widget

# consumer_container = consumers_widget.ConsumerTable()

# consumer_step_number = 5 # change
# consumer_step = v.StepperStep(
#     step=consumer_step_number, children=["Consumers"], complete=False
# )
# consumer_step_content = v.StepperContent(
#     step=consumer_step_number,
#     children=[
#         consumer_container,
#         BackBtn(),
#         NextBtn(consumer_container, consumer_step,disabled=False),
#     ],
# )


In [13]:
import usage_widget
usage_container = usage_widget.Usage()

usage_step_number = 4
usage_step = v.StepperStep(
    step=usage_step_number, children=["Consumers"], complete=False
)
usage_step_content = v.StepperContent(
    step=usage_step_number,
    children=[
        usage_container,
        BackBtn(),
        NextBtn(usage_container, usage_step),
    ],
)

In [14]:
def get_left_over_power(production, usage):
    return production - usage

In [26]:
def collect_data():
    battery = battery_container.battery
    production = pv_container.pv_power_data
    usage = usage_container.usage_profile
    return battery, production, usage
b, p, u =collect_data()

hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
day,,,,,,,,,,,,,,,,,,,,,
1,281.962347,218.520819,177.755178,169.941764,171.555404,175.462111,216.397609,316.273428,436.107425,529.104043,...,533.944963,504.050160,491.990324,506.003513,565.623262,612.758534,587.364937,522.564555,504.814515,423.453091
2,303.486456,225.751923,184.513669,171.388557,169.271603,165.207052,173.844223,235.405231,380.882277,542.786882,...,488.000125,421.273750,367.079740,371.059612,428.979461,514.927774,539.992503,509.762407,474.875013,366.063602
3,248.568501,185.159892,168.104581,158.310442,164.811724,200.104398,330.636634,438.118870,478.308614,486.582973,...,445.042313,401.137551,387.628393,413.717954,480.166123,557.759348,562.571985,532.007516,488.018321,378.509712
4,247.857565,184.630312,167.623781,157.857654,164.340342,199.532075,329.690973,436.865796,476.940592,485.191286,...,443.769437,399.990248,386.519728,412.534669,478.792789,556.164087,560.962960,530.485909,486.622529,377.427127
5,247.159771,184.110522,167.151870,157.413237,163.877674,198.970332,328.762793,435.635887,475.597861,483.825326,...,442.520092,398.864155,385.431558,411.373260,477.444843,554.598317,559.383680,528.992431,485.252539,376.364556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,254.712407,189.736518,172.259647,162.223425,168.885400,205.050409,338.809030,448.947921,490.131041,498.609918,...,456.042492,411.052529,397.209463,423.943883,492.034462,571.545570,576.477162,545.157226,500.080744,387.865394
362,253.901404,189.132398,171.711174,161.706907,168.347670,204.397530,337.730265,447.518474,488.570467,497.022348,...,454.590456,409.743740,395.944751,422.594049,490.467828,569.725773,574.641663,543.421450,498.488491,386.630433
363,308.124650,229.202097,187.333597,174.007894,171.858587,167.731917,176.501090,239.002937,386.703313,551.082311,...,495.458247,427.712090,372.689832,376.730529,435.535568,522.797432,548.245226,517.553122,482.132543,371.658164


In [50]:
import statistics
def get_final_graph():
    b,p,u = collect_data()
    p_x=[day for day in p.index]
    p_y =[sum(day) for day in p.values]
    u_x=[day for day in u.index]
    u_y =[sum(day) for day in u.values]
    plot = ipywidgets.Output()
    with plot as out:
        fig, ax = plt.subplots(figsize=(15, 5))
        ax.plot(p_x, p_y,"g",label="production")
        ax.plot(u_x,u_y,"r",label="consum")
        plt.legend()
        plt.show()
    return plot

get_final_graph()

Output()

In [49]:
stepper = v.Stepper(
    v_model=1,
    children=[
        v.StepperHeader(
            children=[
                address_step,
                v.Divider(),
                battery_step,
                v.Divider(),
                pv_step,
                v.Divider(),
                usage_step
            ]
        ),
        v.StepperItems(
            children=[
                address_step_content,
                battery_step_content,
                pv_step_content,
                usage_step_content,
            ]
        ),
    ],
)
stepper


Stepper(children=[StepperHeader(children=[StepperStep(children=['Address'], complete=True, step=1.0), Divider(…